In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path

['/Users/gsg/Workspace/src/WriteNow/api',
 '/opt/homebrew/Cellar/python@3.11/3.11.8/Frameworks/Python.framework/Versions/3.11/lib/python311.zip',
 '/opt/homebrew/Cellar/python@3.11/3.11.8/Frameworks/Python.framework/Versions/3.11/lib/python3.11',
 '/opt/homebrew/Cellar/python@3.11/3.11.8/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload',
 '',
 '/Users/gsg/Library/Caches/pypoetry/virtualenvs/api-PQsLHg-7-py3.11/lib/python3.11/site-packages']

In [3]:
import pandas as pd
from textwrap import dedent

In [4]:
from app.preprocessing import dropna
from app.database import pool

sql = dedent("""\
select b.title, bss.score, bss.record_date
from books b
    join books_in_bookstores bib on b.id = bib.book_id
    join book_categories_of_books bcib on b.id = bcib.book_id
    join book_categories bc on bcib.category_id = bc.id
    join bookstores bs on bib.store_id = bs.id
    join book_sales_scores bss on b.id = bss.book_id and bs.id = bss.store_id
where bs.alias = 'aladin'
  and bc.name = '컴퓨터/모바일'
""")
with pool.get_connection() as conn:
    data = pd.read_sql(sql, conn)
data = dropna(data)

/var/folders/0n/cy053mms5j12szkdzf21ysm40000gn/T/ipykernel_33815/3486125102.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql, conn)


In [5]:
from app.preprocessing import without_outlier_over_Q1_Q3
data = without_outlier_over_Q1_Q3(data, 'score')
data.head(10)

,title,score,record_date
0,비전공자 마케터로 살아남기,492,2024-04-23
1,어쩌다 마케터를 위한 AI 활용법,72,2024-04-23
2,이모티콘 작가 되기 with 프로크리에이트,600,2024-04-23
3,챗GPT 프롬프트 120% 질문 기술,396,2024-04-23
4,그림으로 이해하는 알고리즘,1068,2024-04-23
5,"대격변 AI 시대, 데이터로 사고하고 데이터로 리드하라",396,2024-04-23
6,코딩 테스트 합격자 되기 : C++ 편,216,2024-04-23
7,줄리아 프로그래밍,312,2024-04-23
8,그리고 싶은 나만의 방과 배경,720,2024-04-23
9,네이버 스마트스토어 시작하기,1205,2023-06-01


In [6]:
from app.preprocessing import remove_stopwords

data['record_date'] = data['record_date'].apply(lambda x: x.strftime('%Y년 %m월'))
data['text'] = data['title'] + ' ' + data['record_date']
data['text']

0                                  비전공자 마케터로 살아남기 2024년 04월
1                              어쩌다 마케터를 위한 AI 활용법 2024년 04월
2                         이모티콘 작가 되기 with 프로크리에이트 2024년 04월
3                            챗GPT 프롬프트 120% 질문 기술 2024년 04월
4                                  그림으로 이해하는 알고리즘 2024년 04월
                                ...                        
110479                               AutoCAD 2024 2024년 04월
110480                       빅라플 ITQ MS Word 2021 2024년 04월
110481    [세트] 직장인 1인의 몫 + 실무에 바로 쓰는 일잘러의 보고서 작성법 - 전2권 ...
110482                         정보통신설비기준 선로설비기준 이해 2024년 04월
110483                      시작해 C 언어 배열과 함수 (워크북) 2024년 04월
Name: text, Length: 99838, dtype: object

In [7]:
data = data.dropna(subset=['text'])

In [10]:
from app.preprocessing import word2vec_vectorize, text_to_vector

X_, vectorizer = word2vec_vectorize(data, 'text', vector_size=5)

In [11]:
X = data['title'].apply(lambda s: text_to_vector(s, model=vectorizer, vector_size=vectorizer.vector_size))
X = pd.DataFrame(X.tolist())
X

,0,1,2,3,4
0,0.918701,0.275527,-0.656989,-2.447692,2.462615
1,1.237711,-0.727338,0.070269,-1.433196,2.459824
2,-0.601680,0.819519,0.848226,-4.132896,-1.948080
3,0.715426,-1.372136,-0.146471,-2.756277,1.868628
4,2.238392,3.208333,-2.841100,-1.416600,2.724158
...,...,...,...,...,...
99833,3.975094,-3.361310,0.110514,-1.420707,-2.334311
99834,3.460990,-1.996819,-0.746453,-1.849135,-2.202195
99835,2.278747,-0.724860,0.682954,-1.339738,0.166170
99836,1.525413,-1.550675,0.585811,-1.137348,0.640365


In [12]:
vectorizer.save("book_title_word2vec.model")

In [13]:
# 판매 지수를 타겟 변수로 설정
y = data['score']

In [14]:
pd.DataFrame(vectorizer.wv.index_to_key)

,0
0,2024년
1,2023년
2,04월
3,05월
4,한
...,...
9057,Googlepedia
9058,디어
9059,글피
9060,Programmers


In [15]:
vectorizer.wv.most_similar('AI', topn=10)

[('거만한', 0.9938002228736877),
 ('덮', 0.991077184677124),
 ('금속', 0.9899720549583435),
 ('전공자', 0.9824780821800232),
 ('양띵', 0.9801652431488037),
 ('눈높이', 0.9719191193580627),
 ('Tableau', 0.9708325266838074),
 ('자동', 0.9689276814460754),
 ('적용', 0.9648333787918091),
 ('아프리카', 0.9647519588470459)]

In [18]:
from app.ml import random_forest_regressor

model, mse, r2 = random_forest_regressor(X, y, 42)
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 8669.78
R-squared: 0.94


In [19]:
from joblib import dump, load
dump(model, 'book_score_model.joblib') 

['book_score_model.joblib']